# Intall Libraries, Run utlity functions, and import the queries (raw topics)

In [ ]:
%pip install -U spacy
%pip install https://huggingface.co/kormilitzin/en_core_med7_trf/resolve/main/en_core_med7_trf-any-py3-none-any.whl
import spacy
import numpy as np
import pandas as pd

print('version',spacy.__version__)

## Example

In [ ]:
nlp = spacy.load("en_core_med7_trf")
text = 'A patient was prescribed Magnesium hydroxide 400mg/5ml suspension PO of total 30ml bid for the next 5 days.'
doc = nlp(text)

for ent in doc.ents:
  print(ent.text, ent.label_)

## Select the collection and its related topics to create the reformulated topics

In [ ]:
collection = 'trec_clc' #trec_clc for trec2021 clinical or cds_clinical for the remaining 3 collections as they share the same topics

if collection == 'trec_clc':
  path_to_topics = './experiments/topics/trec_clc/topics2021.txt'
  path_to_save_entities = './experiments/topics/trec_clc/extracted_med_entities/'

elif collection == 'cds_clinical':
  path_to_topics = './topics-2014_2015-description.topics'
  path_to_save_entities = './experiments/topics/cds_clinical/extracted_med_entities/'

else:
  raise Exception("Invalid selection of topics. Use appropriate collections (trec_clc for TREC 2021 clinical. cds_clinical for cds and clinical collections)")

## Loading the associated topics

In [ ]:
# getting trec 2021 Data
def get_trec(path):
  topics = pd.read_csv(path)
  return topics

# parsing clinical and cds data
def parse_xml(path):
  with open(path, 'r') as document:
    d = document.read()
    soup = BeautifulSoup(d, 'xml')
    qid = soup.find_all('NUM')
    query = soup.find_all('TITLE')
    lq = []
    for i in qid: 
      lq.append(i.text)
    ld = []
    for x in query: 
      ld.append(x.text)
  desc_zu = pd.DataFrame({'qid': lq,'query': ld})
  return desc_zu

if collection =='trec_clc':
  topics = get_trec('/content/drive/MyDrive/data_colab/table_topics2021.csv')
  topics.head()
elif collection == 'cds_clinical':
  topics = parse_xml('/content/drive/MyDrive/data_colab/topics-2014_2015-description.topics')
  topics.head()

# Med7 implementation

In [ ]:
def query_to_ne(query,ent_type):
  """
  extracts the necessary entities from the query
  """
  l = []
  proc = nlp(query)
  for ent in proc.ents:
    if ent.label_ == ent_type:
      l.append(ent.text)
  return l


top1 = pd.DataFrame(columns=["TOPIC","DRUG","DOSAGE","DURATION","FORM","FREQUENCY","ROUTE","STRENGTH"])
#print(topics.head())
list_topics = topics['query'].to_list()
#print(list_topics)
count = 0
d= {"TOPIC":0,"DRUG":[],"DOSAGE":[],"DURATION":[],"FORM":[],"FREQUENCY":[],"ROUTE":[],"STRENGTH":[]}
for topic in list_topics:
  count +=1
  d['TOPIC']= count
  proc = med7(topic)
  for ent in proc.ents:
    if ent.label_ in d:
      d[ent.label_].append(ent.text)
  top1 = top1.append(d, ignore_index=True)
  d= {"TOPIC":0,"DRUG":[],"DOSAGE":[],"DURATION":[],"FORM":[],"FREQUENCY":[],"ROUTE":[],"STRENGTH":[]}


In [ ]:
top1['DOSAGE'] = top1['DOSAGE'].apply(lambda y: np.nan if len(y)==0 else y)
top1['DURATION'] = top1['DURATION'].apply(lambda y: np.nan if len(y)==0 else y)
top1['FORM'] = top1['FORM'].apply(lambda y: np.nan if len(y)==0 else y)
top1['FREQUENCY'] = top1['FREQUENCY'].apply(lambda y: np.nan if len(y)==0 else y)
top1['ROUTE'] = top1['ROUTE'].apply(lambda y: np.nan if len(y)==0 else y)
top1['STRENGTH'] = top1['STRENGTH'].apply(lambda y: np.nan if len(y)==0 else y)
print(top1.head())

In [ ]:
path_to_save_entities = path_to_save_entities+'chem_med7'+'.csv'
top1.to_csv(path_to_save_entities,index=False)